In [1]:
import os
import pyspark

conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')

sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/13 05:35:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/opt/conda/envs/bigdata-fall22/lib/python3.7/site-packages/pyspark/sql/context.py:159: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


### Q1

In [2]:
BreadBasket = spark.read.option("inferSchema", "true").option("header", "true").csv("shared/hw2/BreadBasket_DMS.csv")

In [3]:
from pyspark.sql.functions import avg, col, desc, hour
BreadBasket.select("Item","Transaction","Time",hour(col("Time")))

Item,Transaction,Time,hour(Time)
Bread,1,2022-10-13 09:58:11,9
Scandinavian,2,2022-10-13 10:05:34,10
Scandinavian,2,2022-10-13 10:05:34,10
Hot chocolate,3,2022-10-13 10:07:57,10
Jam,3,2022-10-13 10:07:57,10
Cookies,3,2022-10-13 10:07:57,10
Muffin,4,2022-10-13 10:08:41,10
Coffee,5,2022-10-13 10:13:03,10
Pastry,5,2022-10-13 10:13:03,10
Bread,5,2022-10-13 10:13:03,10


In [4]:
#Using Python
#BreadBasket_filter.where(.groupby("Item").count().sort(desc("count")).show(5) Have to add hours
 

#Using SQL
BreadBasket.createOrReplaceTempView("BreadBasket")
maxSql = spark.sql(""" 
SELECT Item, count(Transaction)
FROM BreadBasket 
WHERE date_part('HOUR', Time) >= 9 and date_part('HOUR', Time) < 23
GROUP BY Item
ORDER BY count(Transaction) DESC
LIMIT 5 """)
maxSql.show()

+------+------------------+
|  Item|count(Transaction)|
+------+------------------+
|Coffee|              5259|
| Bread|              3151|
|   Tea|              1414|
|  Cake|              1017|
|Pastry|               797|
+------+------------------+



### Q2

In [5]:
Restaurants = spark.read.option("inferSchema", "true").option("header", "true").csv("shared/hw2/Restaurants_in_Durham_County_NC.csv",sep=';')

In [6]:
Restaurants

ID,Premise_Name,Premise_Address1,Premise_Address2,Premise_City,Premise_State,Premise_Zip,Premise_Phone,Hours_Of_Operation,Opening_Date,Closing_Date,Seats,Water,Sewage,Insp_Freq,Est_Group_Desc,Risk,Smoking_Allowed,Type_Description,Rpt_Area_Desc,Status,Transitional_Type_Desc,geolocation
56060,WEST 94TH ST PUB,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,NC,27707,(919) 403-0025,null,1994-09-01,null,60,5 - Municipal/Com...,3 - Municipal/Com...,4,Full-Service Rest...,4,NO,1 - Restaurant,Food Service,ACTIVE,FOOD,"35.9207272, -78.9..."
58123,BROOKDALE DURHAM IFS,4434 BEN FRANKLIN...,null,DURHAM,NC,27704,(919) 479-9966,null,2003-10-15,null,350,5 - Municipal/Com...,3 - Municipal/Com...,4,Nursing Home,4,NO,16 - Institutiona...,Food Service,ACTIVE,FOOD,"36.0467802, -78.8..."
70266,SMOOTHIE KING,1125 W. NC HWY 54...,null,DURHAM,NC,27707,(919) 489-7300,null,2009-07-09,null,7,5 - Municipal/Com...,3 - Municipal/Com...,2,Fast Food Restaurant,2,NO,1 - Restaurant,Food Service,ACTIVE,FOOD,"35.9182655, -78.9..."
97837,HAMPTON INN & SUITES,1542 N GREGSON ST,null,DURHAM,NC,27701,(919) 688-8880,null,2012-01-09,null,100,5 - Municipal/Com...,3 - Municipal/Com...,2,Full-Service Rest...,2,NO,1 - Restaurant,Food Service,ACTIVE,FOOD,"36.0183378, -78.9..."
60690,BETTER LIVING CON...,909 GARCIA ST,null,DURHAM,NC,27704,(919) 477-5825,null,2008-06-02,null,6,5 - Municipal/Com...,3 - Municipal/Com...,1,null,0,N/A,43 - Residential ...,Residential Care,ACTIVE,N/A,"36.0556347, -78.9..."
60686,ADVENTURE HOUSE,4 KIMBROUGH COURT,null,DURHAM,NC,27703,(919) 957-9097,null,2008-06-02,null,0,5 - Municipal/Com...,3 - Municipal/Com...,1,null,0,N/A,43 - Residential ...,Residential Care,ACTIVE,N/A,"35.984012, -78.80..."
85252,ANOTHER BEAUTIFUL...,1309 ANGIER AVE.,null,DURHAM,NC,27701,(919) 682-5292,null,2010-08-23,null,null,5 - Municipal/Com...,3 - Municipal/Com...,2,null,0,NO,42 - Child Care,Day Care,ACTIVE,N/A,"35.9857413, -78.8..."
59120,BRIDGES AT SOUTHP...,7304 CALIBRE PARK DR,null,DURHAM,NC,27707,null,null,1996-04-02,null,0,5 - Municipal/Com...,3 - Municipal/Com...,2,null,0,N/A,53 - Year-Round S...,Swimming Pools,ACTIVE,N/A,"35.913596, -78.96..."
59124,SHEARTON INN UNIV...,2800 CAMPUS WALK AVE,null,DURHAM,NC,27705,null,null,1996-04-25,null,0,5 - Municipal/Com...,3 - Municipal/Com...,2,null,0,N/A,53 - Year-Round S...,Swimming Pools,ACTIVE,N/A,"36.0111429, -78.9..."
59263,SPA HEALTH CLUB,3419 HILLSBOROUGH RD,null,DURHAM,NC,27705,null,null,2000-05-30,null,0,5 - Municipal/Com...,3 - Municipal/Com...,2,null,0,N/A,55 - Year-Round Spa,Swimming Pools,ACTIVE,N/A,"36.0184133, -78.9..."


In [7]:
print(Restaurants.select("Id").count())
print("Shows that ID is the primary key as no of rows in df and id count is same")

2472
Shows that ID is the primary key as no of rows in df and id count is same


Method 1 

In [8]:
Restaurants.select("Rpt_Area_Desc").groupby("Rpt_Area_Desc").count().sort(desc("count"))

Rpt_Area_Desc,count
Food Service,1093
Swimming Pools,420
Summer Food,242
Day Care,173
Residential Care,154
Mobile Food,147
School Buildings,89
Lodging,62
Tattoo Establishm...,32
Institutions,30


Method 2 

In [9]:
Restaurants.createOrReplaceTempView("Restaurants")
maxSql = spark.sql(""" 
SELECT Rpt_Area_Desc, count(Id)
FROM Restaurants 
GROUP BY Rpt_Area_Desc
ORDER BY count(Id) desc
""")
maxSql.show()

+--------------------+---------+
|       Rpt_Area_Desc|count(Id)|
+--------------------+---------+
|        Food Service|     1093|
|      Swimming Pools|      420|
|         Summer Food|      242|
|            Day Care|      173|
|    Residential Care|      154|
|         Mobile Food|      147|
|    School Buildings|       89|
|             Lodging|       62|
|Tattoo Establishm...|       32|
|        Institutions|       30|
|                null|       13|
|      Adult Day Care|        5|
|  Bed&Breakfast Home|        4|
|        Summer Camps|        4|
|   Local Confinement|        2|
|   Bed&Breakfast Inn|        2|
+--------------------+---------+



### Q3

In [10]:
Population = spark.read.option("inferSchema", "true").option("header", "true").csv("populationbycountry19802010millions.csv")
Population = Population.dropna()
Population

22/10/13 05:36:19 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


_c0,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010
North America,320.27638,324.44694,328.62014,332.72487,336.72143,340.74811,344.89548,349.07829,353.2939,357.68457,362.4468,367.70684,373.29069,378.74233,383.9166,388.97216,393.9428,398.97205,403.85585,408.60296,413.3245,417.83236,422.05268,426.06238,430.26938,434.47232,438.82964,443.3473,447.67394,451.83698,456.59331
Bermuda,0.05473,0.05491,0.05517,0.05551,0.05585,0.05618,0.05651,0.05683,0.05717,0.05749,0.05778,0.0581,0.0587,0.05924,0.05975,0.06029,0.06087,0.06145,0.06198,0.06251,0.06306,0.06361,0.06418,0.06476,0.06534,0.06591,0.06644,0.06692,0.06739,0.06784,0.06827
Canada,24.5933,24.9,25.2019,25.4563,25.7018,25.9416,26.2038,26.5497,26.8948,27.3793,27.7906,28.1179,28.54489,28.95334,29.33081,29.69053,30.02632,30.3056,30.55166,30.82026,31.09956,31.37674,31.64096,31.88931,32.13476,32.38638,32.65668,32.93596,33.2127,33.48721,33.75974
Greenland,0.05021,0.05103,0.05166,0.05211,0.05263,0.05315,0.05364,0.0541,0.05485,0.05541,0.05563,0.05554,0.05549,0.05564,0.05592,0.05619,0.05634,0.05651,0.05661,0.0567,0.05689,0.05713,0.05736,0.05754,0.0577,0.05778,0.05764,0.05753,0.05756,0.0576,0.05764
Mexico,68.34748,69.96926,71.6409,73.36288,75.08014,76.76723,78.44243,80.12249,81.78182,83.36684,84.91365,86.48803,88.11103,89.74914,91.3379,92.88035,94.39858,95.89515,97.32506,98.61691,99.92662,101.24696,102.47993,103.71806,104.95959,106.2029,107.44953,108.70089,109.9554,111.21179,112.46886
Saint Pierre and ...,0.00599,0.00601,0.00605,0.00607,0.00611,0.00616,0.00621,0.00625,0.00628,0.00631,0.00632,0.00633,0.00636,0.00638,0.0064,0.0064,0.00641,0.00642,0.00642,0.00643,0.00641,0.00637,0.00633,0.00629,0.00625,0.0062,0.00615,0.0061,0.00605,0.006,0.00594
United States,227.22468,229.46571,231.66446,233.79199,235.8249,237.9238,240.13289,242.28892,244.49898,246.81923,249.62281,252.98094,256.51422,259.91859,263.12582,266.27839,269.39428,272.64693,275.8541,279.04017,282.17196,285.08156,287.80391,290.32642,293.04574,295.75315,298.59321,301.5799,304.37485,307.00655,310.23286
Central & South A...,293.05856,299.43033,305.95253,312.51136,318.87955,325.22704,331.82291,338.59859,345.44544,352.20471,358.79973,365.15137,371.43224,377.7438,384.26984,390.75665,397.13002,403.41352,409.62879,415.63607,421.4539,427.24012,433.05116,438.97976,445.01525,451.05504,457.01699,462.89157,468.73872,474.53897,480.01228
Antarctica,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Antigua and Barbuda,0.06855,0.06826,0.06801,0.06562,0.06447,0.0644,0.0644,0.06442,0.06443,0.06439,0.06416,0.06403,0.06518,0.06633,0.06749,0.06871,0.07006,0.07145,0.07278,0.07402,0.07535,0.07673,0.07794,0.07907,0.08019,0.08128,0.08234,0.08343,0.08452,0.08563,0.08675


Method 1

In [11]:
from pyspark.sql.functions import udf,lit, cast, col
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType

data = []


for k in Population.columns[1:]:
    Population = Population.withColumn(k,col(k).cast('double'))

for i,j in zip(Population.columns[1:], Population.columns[2:]):
    Population = Population.withColumn("Percentage_increase("+i+","+j+")",((col(j)-col(i))*100)/col(i)) 
                                                

In [12]:
Population = Population.dropna() 

In [ ]:
from pyspark.sql.functions import udf,lit, cast, col, asc, desc
for i,j in zip(Population.columns[1:], Population.columns[2:]):
    if len(i)==4 and len(j)==4:
        print(j)
        col_name = "Percentage_increase("+i+","+j+")"
        Population = Population.sort(desc(col_name))
        print("MAX",Population.collect()[0]["_c0"], Population.collect()[0][col_name])
        Population = Population.sort(asc(col_name))
        print("MIN",Population.collect()[0]["_c0"], Population.collect()[0][col_name])

1981


Method 2

In [ ]:
Population = spark.read.option("inferSchema", "true").option("header", "true").csv("populationbycountry19802010millions.csv")
Population.dropna()

In [ ]:
from pyspark.sql.functions import expr
expression = "stack(31, '1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, '1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, '1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, '1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, '2000', `2000`, '2001', `2001`, '2002', `2002`, '2003', `2003`, '2004', `2004`, '2005', `2005`, '2006', `2006`, '2007', `2007`, '2008', `2008`, '2009', `2009`, '2010', `2010`) as (Year, Population)"
Population_unpivot = Population.select("_c0", expr(expression))
Population_unpivot.show()

In [ ]:
from pyspark.sql.window import Window
windowSpec  = Window.partitionBy("_c0").orderBy("Year")

In [ ]:

from pyspark.sql.functions import lag
Population_unpivot_lag = Population_unpivot.withColumn('LastYear_Population',lag(Population_unpivot['Population']).over(windowSpec))

Population_unpivot_lag.dropna()

In [ ]:
from pyspark.sql.types import FloatType

def percentage_increase(a,b): 
    return ((b-a)*100)/a


percentage_increase_udf = udf(percentage_increase, FloatType())
Population_unpivot_percent_growth = Population_unpivot_lag.withColumn("Percentage_increase",  ((Population_unpivot_lag['Population'] - Population_unpivot_lag['LastYear_Population']) / Population_unpivot_lag['LastYear_Population'])*100)                                                           
                                                                                                              

In [ ]:
Population_unpivot_percent_growth = Population_unpivot_percent_growth.dropna()

In [ ]:
from pyspark.sql.functions import col, dense_rank, desc, asc

max_increase_window = Window.partitionBy("Year").orderBy(desc("Percentage_increase"))
max_increase = Population_unpivot_percent_growth.withColumn("Max", dense_rank().over(max_increase_window)).where(col("Max") == 1)


max_increase = max_increase.withColumn("Type", lit("Max"))

min_increase_window = Window.partitionBy("Year").orderBy(asc("Percentage_increase"))
min_increase = Population_unpivot_percent_growth.withColumn("Min", dense_rank().over(min_increase_window)).where(col("Min") == 1)
min_increase = min_increase.withColumn("Type", lit("Min"))

min_increase.union(max_increase).select("_c0", "Year","Type", "Percentage_increase").orderBy("Year").show()
min_increase.union(max_increase).select("_c0", "Year","Type", "Percentage_increase").orderBy(asc("Year"),desc("Type")).show()

Q4

In [ ]:
df = spark.read.text("shared/hw2/romeo-juliet-pg1777.txt")

In [ ]:
df

In [ ]:
from pyspark.sql.functions import regexp_replace, regexp_extract, lower, explode, split, col, desc
df = df.withColumn("all_lower",lower(col('value')))

In [ ]:
df1 = df.withColumn("parsed",regexp_replace(col("all_lower"),'[^a-z0-9]',' '))

In [ ]:
df1.select(explode(split(col("parsed"),' ')).alias("exploding")).filter(col("exploding")!='').groupby("exploding").count().sort(desc("count"))

### Q5

In [ ]:
json_reader = spark.read.format("json").load("shared/hw2/durham-nc-foreclosure-2006-2016.json")
json_reader

In [ ]:
Restaurants = spark.read.option("inferSchema", "true").option("header", "true").csv("shared/hw2/Restaurants_in_Durham_County_NC.csv",sep=';')
Restaurants.columns

In [ ]:
Restaurant_active = Restaurants.filter((Restaurants.Status=="ACTIVE") & (Restaurants.Rpt_Area_Desc=="Food Service"))

In [ ]:
Restaurant_active.groupby(col("Status")).count()

In [ ]:
Restaurant_subset = Restaurant_active.select("id","geolocation","Premise_City")
Restaurant_subset = Restaurant_subset.na.drop(subset=["geolocation"])
Restaurant_two = Restaurant_subset.select("*")
Restaurant_one = Restaurant_subset.withColumnRenamed("geolocation","geolocation_x")
Restaurant_one = Restaurant_one.withColumnRenamed("id","id_x")
Restaurant_one = Restaurant_one.withColumnRenamed("Premise_City","Premise_City_x")

In [ ]:
Restaurant_two

In [ ]:
Restaurant_one = Restaurant_one.join(Restaurant_two, ((Restaurant_one["id_x"]!=Restaurant_two["id"]) & (Restaurant_one["Premise_City_x"]==Restaurant_two["Premise_City"])), "cross")
Restaurant_one



In [ ]:
from haversine import haversine, Unit
def haversine_distance(a,b):
    a_lis = list(a.split(","))
    li = [float(x) for x in a_lis]

    b_lis = list(b.split(","))
    lj = [float(x) for x in b_lis]
    
    distance = haversine(li, lj)
    
    return distance

haversine_distance_udf = udf(haversine_distance,DoubleType())

In [ ]:
'''
a = Restaurant_one.select("geolocation").collect()[0]
print(a)
b = Restaurant_one.select("geolocation_x").collect()[0]
print(b)
haversine_distance(a[0],b[0])
'''

In [ ]:
Restaurant_one

In [ ]:
Restaurant_one.withColumn("distance", haversine_distance_udf(col("geolocation_x"),col("geolocation"))).sort("distance").show()
